## Preprocesing ML Data
These sets of code processes the source file "United_Kingdom.h5" into the train-test-val csv files for the baseline, 5km median aggregation and 10-nn scenarios respectively.

⚠️ *Note*: These h5 files may take a LONG TIME TO PROCESS due to the nature of the file type and number of image bands within (>250,000). Alternatively, skip this step as the output files are saved nicely in the github repo for use.

To sum up ...

Input file: 

        United_Kingdom.h5

Output files:

    Baseline (no spatial data) scenario:

        uk_monthly_test_norm.csv

        uk_monthly_train_norm.csv

        uk_monthly_val_norm.csv

    10-Nearest Neighbours scenario:

        uk_monthly_test_10nn_norm.csv

        uk_monthly_train_10nn_norm.csv

        uk_monthly_val_10nn_norm.csv

    5km Median Aggregation scenario:

        uk_monthly_test_adj_norm.csv

        uk_monthly_train_adj_norm.csv
        
        uk_monthly_val_adj_norm.csv

#### Split the h5 into train-test-split

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Crop image stack to desired size, preserving channels and time
def crop_center(img_b, cropx, cropy):
    c, t, y, x = img_b.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)
    return img_b[:, :, starty:starty + cropy, startx:startx + cropx]

# Calculate median composite image for a given month
def calculate_monthly_composite(image, indices):
    monthly_stack = np.stack([image[idx] for idx in indices])
    return np.median(monthly_stack, axis=0)

from sklearn.model_selection import train_test_split

# Open the HDF5 file and get dataset keys
with h5py.File(file_path, "r") as h5_file:
    dataset_keys = list(h5_file.keys())  # Get all dataset names (keys)

# Shuffle dataset keys for randomness
np.random.shuffle(dataset_keys)

# Split dataset keys into train (70%), test (15%), validation (15%)
train_keys, temp_keys = train_test_split(dataset_keys, test_size=0.3, random_state=42)
test_keys, val_keys = train_test_split(temp_keys, test_size=0.5, random_state=42)

# Save split datasets into new HDF5 files (with attributes)
def save_hdf5(file_name, file_path, dataset_keys):
    with h5py.File(file_name, "w") as new_h5:
        with h5py.File(file_path, "r") as original_h5:
            for key in dataset_keys:
                # Copy dataset
                data = original_h5[key][()]
                dataset = new_h5.create_dataset(key, data=data, compression="gzip")

                # Copy attributes
                for attr_name, attr_value in original_h5[key].attrs.items():
                    dataset.attrs[attr_name] = attr_value  # Copy attribute value
                
    print(f"Saved {len(dataset_keys)} datasets to {file_name}")


file_path = "United_Kingdom.h5"
# Save each split as a new HDF5 file
save_hdf5("uk_train.h5", file_path, train_keys)
save_hdf5("uk_test.h5", file_path, test_keys)
save_hdf5("uk_val.h5", file_path, val_keys)

#### Saves out the Baseline Scenario (no spatial data) CSV files

In [1]:
import os
import h5py
import numpy as np
import pandas as pd

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

def crop_center(img_b, cropx, cropy):
    c, t, y, x = img_b.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)
    return img_b[:, :, starty:starty + cropy, startx:startx + cropx]

def calculate_monthly_composite(image, indices):
    monthly_stack = np.stack([image[idx] for idx in indices])
    return np.median(monthly_stack, axis=0)

def process_split(h5_file_path, split_name):
    print(f"Processing {split_name}...")

    with h5py.File(h5_file_path, 'r') as f:
        keys = list(f.keys())
        monthly_data = {month: {band: [] for band in bands} for month in months}
        Lc1, Lu1, Lucas_id = [], [], []

        for key in keys:
            im = f[key]
            mask = im['SCL'] < 9

            image = np.stack([np.where(mask == 1, im[band], 0) for band in bands], axis=0)
            image = np.moveaxis(image, [0], [1])

            # Month-wise indices
            image_id_list = list(im.attrs['Image_ID'])
            month_indices = {
                month: [i for i, s in enumerate(image_id_list) if f'2018{str(idx+1).zfill(2)}' in s]
                for idx, month in enumerate(months)
            }

            monthly_composites = []
            for month in months:
                if month_indices[month]:
                    composite = calculate_monthly_composite(image, month_indices[month])
                else:
                    composite = np.zeros_like(image[0])
                monthly_composites.append(composite)

            stacked_image = np.stack(monthly_composites, axis=0)
            cropped_image = crop_center(stacked_image, 3, 3)

            for idx, month in enumerate(months):
                for band_idx, band in enumerate(bands):
                    monthly_data[month][band].append(cropped_image[idx][band_idx][1][1])

            Lc1.append(im.attrs.get('lc1', 'Unknown'))
            Lu1.append(im.attrs.get('lu1', 'Unknown'))
            Lucas_id.append(key[12:])

    # Save to CSV
    data = {'Lucas_ID': Lucas_id}
    for month in months:
        for band in bands:
            data[f'{month}_{band}'] = monthly_data[month][band]
    data['Lc1'] = Lc1
    data['Lu1'] = Lu1

    df = pd.DataFrame(data)
    output_path = os.path.join(f'uk_monthly_{split_name}.csv')
    df.to_csv(output_path, index=False)
    print(f"Saved CSV to {output_path}")

# Run for all splits
if __name__ == "__main__":
    for split in ['train', 'val', 'test']:
        h5_file_path = os.path.join(f'uk_{split}.h5')
        process_split(h5_file_path, split)

Processing train...


KeyboardInterrupt: 

In [ ]:
# Normalization of data

import os
import numpy as np
import pandas as pd


def normalise_data(split):
    file_path = os.path.join(f'uk_monthly_{split}.csv')
    df = pd.read_csv(file_path)

    bands = [col for col in df.columns if '_' in col and col not in ['Lc1', 'Lu1', 'Lucas_ID']]
    df = df.dropna(subset=['Lc1'])

    for band in bands:
        q_low, q_hi = df[band].quantile(0.01), df[band].quantile(0.999)
        df[band] = (df[band] - q_low) / (q_hi - q_low + np.exp(-12))
        df[band] = df[band].clip(0, 1)

    output_path = os.path.join(f'uk_monthly_{split}_norm.csv')
    df.to_csv(output_path, index=False)
    print(f"Saved normalized CSV to {output_path}")

# Run for each split
if __name__ == "__main__":
    for split in ['train', 'val', 'test']:
        normalise_data(split)

#### Saves out the 10-Nearest Neighbours Scenario CSV files

In [2]:
# Creating the adjacency matrix for the UK dataset using the 10 nearest neighbors
import h5py
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

def extract_coordinates(h5_file):
    """Extracts Lucas point coordinates from the HDF5 file."""
    coords = []
    ids = []
    
    with h5py.File(h5_file, 'r') as f:
        for key in f.keys():
            if key.startswith("Lucas_Point_"):
                point_id = key.split("_")[-1]  # Extract ID
                lat, lon = f[key].attrs["Coordinates"]  # (lat, lon)
                coords.append([lat, lon])
                ids.append(point_id)
    
    return np.array(coords), ids

def create_nearest_neighbors(coords, ids, k=10):
    """Finds the k nearest neighbors (excluding self) and their Euclidean distances using KDTree."""
    tree = cKDTree(coords)
    adjacency_list = {id_: [] for id_ in ids}
    
    # Query k+1 because the nearest neighbor is always the point itself
    distances, indices = tree.query(coords, k=k+1)  

    for i, lucas_id in enumerate(ids):
        neighbors = indices[i][1:]  # Exclude self (first index is the point itself)
        dists = distances[i][1:]  # Exclude self distance (0)
        adjacency_list[lucas_id] = [(ids[j], dists[idx]) for idx, j in enumerate(neighbors)]
    
    return adjacency_list

def save_nearest_neighbors(adjacency_list, output_file):
    """Save nearest neighbors list to a CSV file."""
    adjacency_data = []
    for lucas_id, neighbors in adjacency_list.items():
        for neighbor_id, distance in neighbors:
            adjacency_data.append([lucas_id, neighbor_id, distance])

    adjacency_df = pd.DataFrame(adjacency_data, columns=['Lucas_ID', 'Neighbor_ID', 'Distance'])
    adjacency_df.to_csv(output_file, index=False)
    print(f"Nearest neighbors saved to {output_file}")

# File path
h5_file_path = "United_Kingdom.h5"
# Extract coordinates
coords, ids = extract_coordinates(h5_file_path)
# Compute nearest neighbors
adjacency_list = create_nearest_neighbors(coords, ids, k=10)
# Save results
save_nearest_neighbors(adjacency_list, "uk_10nn_matrix.csv")

Nearest neighbors saved to uk_10nn_matrix.csv


In [3]:
# Adapting the train, test, val csv to include the 10-NN band values
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# File paths
train_file = "uk_monthly_train.csv"
test_file = "uk_monthly_test.csv"
val_file = "uk_monthly_val.csv"
adjacency_file = "uk_10nn_matrix.csv"

# Load datasets
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)
val_df = pd.read_csv(val_file)
adjacency_df = pd.read_csv(adjacency_file)

# Merge all datasets
all_data = pd.concat([train_df, test_df, val_df], ignore_index=True)

# Extract band columns
band_columns = [col for col in train_df.columns if col.startswith(('Jan_', 'Feb_', 'Mar_', 'Apr_', 'May_', 'Jun_', 
                                                                   'Jul_', 'Aug_', 'Sep_', 'Oct_', 'Nov_', 'Dec_'))]

# Create a dictionary mapping Lucas_ID to its band values
data_dict = all_data.set_index("Lucas_ID")[band_columns].to_dict(orient="index")

# Create adjacency dictionary with distances
adjacency_df = adjacency_df.sort_values(by=["Lucas_ID", "Distance"])  # Ensure nearest are in order
adj_dict = adjacency_df.groupby("Lucas_ID")["Neighbor_ID"].apply(list).to_dict()

def append_neighbor_bands(df, k=10):
    """Appends band values from the k nearest neighbors."""
    neighbor_bands = {f"{col}_n{n+1}": np.zeros(len(df)) for col in band_columns for n in range(k)}

    for i, lucas_id in enumerate(df["Lucas_ID"]):
        neighbors = adj_dict.get(lucas_id, [])[:k]  # Get up to k neighbors

        for n, neighbor_id in enumerate(neighbors):
            if neighbor_id in data_dict:
                for col in band_columns:
                    neighbor_bands[f"{col}_n{n+1}"][i] = data_dict[neighbor_id][col]  # Assign neighbor band value

    # Append new columns
    for col in band_columns:
        for n in range(k):
            df[f"{col}_n{n+1}"] = neighbor_bands[f"{col}_n{n+1}"]

    return df

# Append nearest neighbor bands for all datasets
train_df = append_neighbor_bands(train_df, k=10)
test_df = append_neighbor_bands(test_df, k=10)
val_df = append_neighbor_bands(val_df, k=10)

# Merge again to ensure normalization is applied consistently
all_data = pd.concat([train_df, test_df, val_df], ignore_index=True)

# Normalize using Min-Max Scaling
scaler = MinMaxScaler()
all_band_cols = band_columns + [f"{col}_n{n+1}" for col in band_columns for n in range(10)]
all_data[all_band_cols] = scaler.fit_transform(all_data[all_band_cols])

# Drop 'Lu1' column if it exists
if 'Lu1' in all_data.columns:
    all_data = all_data.drop(columns=['Lu1'])

# Split datasets back
train_df = all_data.iloc[:len(train_df)]
test_df = all_data.iloc[len(train_df):len(train_df) + len(test_df)]
val_df = all_data.iloc[len(train_df) + len(test_df):]

# Save updated files
train_df.to_csv("uk_monthly_train_10nn_norm.csv", index=False)
test_df.to_csv("uk_monthly_test_10nn_norm.csv", index=False)
val_df.to_csv("uk_monthly_val_10nn_norm.csv", index=False)

print("Updated files saved with normalized 10NN band columns.")

/var/folders/z5/zqtg1_hn7_x7fpg92yxfn09r0000gn/T/ipykernel_4289/2212271272.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_n{n+1}"] = neighbor_bands[f"{col}_n{n+1}"]
/var/folders/z5/zqtg1_hn7_x7fpg92yxfn09r0000gn/T/ipykernel_4289/2212271272.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_n{n+1}"] = neighbor_bands[f"{col}_n{n+1}"]
/var/folders/z5/zqtg1_hn7_x7fpg92yxfn09r0000gn/T/ipykernel_4289/2212271272.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

Updated files saved with normalized 10NN band columns.


#### Saves out the 5km Median Aggregation CSV files

In [4]:
# Creating an adjacency matrix with LUCAS points within 5.0km
import h5py
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

def extract_coordinates(h5_file):
    """Extracts Lucas point coordinates from the HDF5 file."""
    coords = []
    ids = []
    
    with h5py.File(h5_file, 'r') as f:
        for key in f.keys():
            if key.startswith("Lucas_Point_"):
                point_id = key.split("_")[-1]  # Extract ID
                lat, lon = f[key].attrs["Coordinates"]  # (lat, lon)
                coords.append([lat, lon])
                ids.append(point_id)
    
    return np.array(coords), ids

def create_adjacency_matrix(coords, ids, radius=5.0):
    """Create adjacency list of Lucas_IDs within the specified radius using KDTree."""
    # cKDTree for fast nn search - far more efficient than brute force loop!!
    tree = cKDTree(coords)
    adjacency_list = {id_: [] for id_ in ids}
    
    for i, lucas_id in enumerate(ids):
        # Query neighbors within 5km (Earth radius ~6371km, so convert to degrees)
        idx = tree.query_ball_point(coords[i], radius / 6371 * (180 / np.pi))
        adjacency_list[lucas_id] = [ids[j] for j in idx if j != i]  # Exclude self
    
    return adjacency_list

def save_adjacency_matrix(adjacency_list, output_file):
    """Save adjacency list to a CSV file."""
    adjacency_data = []
    for lucas_id, neighbors in adjacency_list.items():
        for neighbor in neighbors:
            adjacency_data.append([lucas_id, neighbor])

    adjacency_df = pd.DataFrame(adjacency_data, columns=['Lucas_ID', 'Neighbor_ID'])
    adjacency_df.to_csv(output_file, index=False)
    print(f"Adjacency matrix saved to {output_file}")


h5_file_path = "United_Kingdom.h5"
# 1: Extract coordinates
coords, ids = extract_coordinates(h5_file_path)
# 2: Compute adjacency matrix 
adjacency_list = create_adjacency_matrix(coords, ids, radius=5.0)
# 3: Save matrix
save_adjacency_matrix(adjacency_list, "uk_adj_matrix.csv")


Adjacency matrix saved to uk_adj_matrix.csv


In [5]:
# Create train, test, val datasets with adjacent median values and normalized band values
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# File paths
train_file = "uk_monthly_train.csv"
test_file = "uk_monthly_test.csv"
val_file = "uk_monthly_val.csv"
adjacency_file = "uk_adj_matrix.csv"

# Load datasets
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)
val_df = pd.read_csv(val_file)
adjacency_df = pd.read_csv(adjacency_file)

# Merge all datasets
all_data = pd.concat([train_df, test_df, val_df], ignore_index=True)

# Extract band columns
band_columns = [col for col in train_df.columns if col.startswith(('Jan_', 'Feb_', 'Mar_', 'Apr_', 'May_', 'Jun_', 
                                                                   'Jul_', 'Aug_', 'Sep_', 'Oct_', 'Nov_', 'Dec_'))]

# Create a dictionary mapping Lucas_ID to its row of band values
data_dict = all_data.set_index("Lucas_ID")[band_columns].to_dict(orient="index")

# Create adjacency dictionary for fast lookup
adj_dict = adjacency_df.groupby("Lucas_ID")["Neighbor_ID"].apply(list).to_dict()

def compute_adjacent_medians(df):
    """Compute median band values of adjacent points using efficient lookups."""
    adj_values = {col: np.zeros(len(df)) for col in band_columns}

    for i, lucas_id in enumerate(df["Lucas_ID"]):
        neighbors = adj_dict.get(lucas_id, [])

        if neighbors:
            neighbor_values = np.array([list(data_dict[n].values()) for n in neighbors if n in data_dict])

            if neighbor_values.size > 0:
                median_vals = np.median(neighbor_values, axis=0)
            else:
                median_vals = np.zeros(len(band_columns))  # Default if no valid neighbors
        else:
            median_vals = np.zeros(len(band_columns))  # Default if no neighbors
        
        for j, col in enumerate(band_columns):
            adj_values[col][i] = median_vals[j]

    # Append new columns
    for col in band_columns:
        df[f"{col}_adj"] = adj_values[col]

    return df

# Compute adjacent medians for all datasets
train_df = compute_adjacent_medians(train_df)
test_df = compute_adjacent_medians(test_df)
val_df = compute_adjacent_medians(val_df)

# Merge again to ensure normalization is applied consistently
all_data = pd.concat([train_df, test_df, val_df], ignore_index=True)

# Normalize using Min-Max Scaling
scaler = MinMaxScaler()
all_data[band_columns + [f"{col}_adj" for col in band_columns]] = scaler.fit_transform(
    all_data[band_columns + [f"{col}_adj" for col in band_columns]]
)

# Drop 'Lu1' column if it exists
if 'Lu1' in all_data.columns:
    all_data = all_data.drop(columns=['Lu1'])

# Split datasets back
train_df = all_data.iloc[:len(train_df)]
test_df = all_data.iloc[len(train_df):len(train_df) + len(test_df)]
val_df = all_data.iloc[len(train_df) + len(test_df):]

# Save normalized files
train_df.to_csv("uk_monthly_train_adj_norm.csv", index=False)
test_df.to_csv("uk_monthly_test_adj_norm.csv", index=False)
val_df.to_csv("uk_monthly_val_adj_norm.csv", index=False)

print("Updated files saved with normalized adjacent median columns.")

/var/folders/z5/zqtg1_hn7_x7fpg92yxfn09r0000gn/T/ipykernel_4289/3183042834.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_adj"] = adj_values[col]
/var/folders/z5/zqtg1_hn7_x7fpg92yxfn09r0000gn/T/ipykernel_4289/3183042834.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_adj"] = adj_values[col]
/var/folders/z5/zqtg1_hn7_x7fpg92yxfn09r0000gn/T/ipykernel_4289/3183042834.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

Updated files saved with normalized adjacent median columns.
